[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Image Processing with Python

## Convolution in Frequency Domain

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 10/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import itertools
import math
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, Literal, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
# General Auxiliary Functions

def Gen2DGaussianKernel( σ: float, kernelRadius: int ) -> np.ndarray:

    vG = np.arange(-kernelRadius, kernelRadius + 1)
    vK = np.exp(-(1 / (2 * σ * σ)) * np.square(vG))

    mK = np.outer(vK, vK)
    mK /= np.sum(mK)

    return mK

def DftConvolution2D( mI: np.ndarray, mH: np.ndarray, /, *, convMode: Literal['full', 'same', 'valid'] ) -> np.ndarray:
    """
    Perform 2D convolution of an input matrix with a kernel using the Discrete Fourier Transform (DFT).

    Parameters
    ----------
    mI : np.ndarray
        Input 2D matrix (Image) to be convolved.
    mH : np.ndarray
        2D kernel matrix (Kernel / Filter) used for convolution.
    convMode : {'full', 'same', 'valid'}
        Convolution mode that determines the shape of the output:
        - 'full': Returns the convolution result with the full extent of overlap between the input and kernel.
        - 'same': Returns the result with the same size as the input, centered appropriately.
        - 'valid': Returns the result only for positions where the kernel fully overlaps the input (no padding).

    Returns
    -------
    np.ndarray
        The resulting matrix after performing the 2D convolution.

    Raises
    ------
    ValueError
        If `mI` or `mH` are not 2D matrices, or if their dimensions are incompatible.

    Notes
    -----
    - This function uses the frequency-domain approach for convolution by computing the Fourier transform
      of the input (`mI`) and kernel (`mH`), multiplying them element wise, and then performing the inverse
      Fourier transform.
    - This method is efficient for large input and kernel sizes but may introduce numerical inaccuracies due
      to rounding in the Fourier transform process.
    - Padding is applied to match the sizes of the input and kernel for the specified `convMode`.

    Examples
    --------
    Convolve a 2D image with a Gaussian kernel:

    >>> import numpy as np
    >>> from scipy.ndimage import gaussian_filter
    >>> mI = np.random.random((100, 100))
    >>> mH = gaussian_filter(np.eye(5), sigma=1.0)
    >>> result = DftConvolution2D(mI, mH, convMode = 'same')
    >>> print(result.shape)
    (100, 100)
    """

    if (np.ndim(mH) != np.ndim(mI)) or (np.ndim(mH) != 2):
        raise ValueError('Both `mK` and `mI` must be a 2D matrix')
    
    numRows     = np.size(mI, 0)
    numCols     = np.size(mI, 1)
    
    numRowsKernel = np.size(mH, 1)
    numColsKernel = np.size(mH, 1)

    match convMode:
        case 'full':
            numRowsFft  = numRows + numRowsKernel - 1
            numColsFft  = numCols + numColsKernel - 1
            firstRowIdx = 1
            firstColIdx = 1
            lastRowIdx  = numRowsFft
            lastColdIdx = numColsFft
        case 'same':
            numRowsFft  = numRows + numRowsKernel
            numColsFft  = numCols + numColsKernel
            firstRowIdx = math.ceil((numRowsKernel + 1) / 2)
            firstColIdx = math.ceil((numColsKernel + 1) / 2)
            lastRowIdx  = firstRowIdx + numRows - 1
            lastColdIdx = firstColIdx + numCols - 1
        case 'valid':
            numRowsFft  = numRows
            numColsFft  = numCols
            firstRowIdx = numRowsKernel
            firstColIdx = numColsKernel
            # The Kernel when transformed is shifted (Namely its (0, 0) is top left not middle).
            lastRowIdx  = numRowsFft
            lastColdIdx = numColsFft
    
    mO = np.fft.ifft2(np.fft.fft2(mI, numRowsFft, numColsFft) * np.fft.fft2(mH, numRowsFft, numColsFft))
    mO = mO[firstRowIdx:(lastRowIdx + 1), firstColIdx:(lastColdIdx + 1)]

    return mO



## Image Convolution in Frequency Domain

The [_Convolution Theorem_](https://en.wikipedia.org/wiki/Convolution_theorem) allows applying [Linear Time Invariant (LTI)](https://en.wikipedia.org/wiki/Linear_time-invariant_system) / Linear Shift Invariant filters in the frequency domain.  
It can be done as Complex Exponentials are the Eigen Functions / Eigen Vectors of such systems.

In te Discrete case, due to the periodicity assumption of the DFT, the applied operation is the [_Circular Convolution_](https://en.wikipedia.org/wiki/Circular_convolution).  
Hence, in order to apply _Linear Convolution_ some padding and indexing operations should be applied.

This notebook examines the performance per implementation.

* <font color='brown'>(**#**)</font> Explicit Frequency 
* <font color='brown'>(**#**)</font> Filters might be neither LPF nor HPF.

In [ ]:
# Parameters

# imgUrl = 'https://i.imgur.com/3BbIXdH.png' #<! A turtle climbing the Everest!
imgUrl = 'https://i.postimg.cc/63rN33GZ/3BbIXdH.png' #<! A turtle climbing the Everest!

lImageSize      = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
lKernelRadius   = [5, 10, 15, 20, 25, 30, 35, 40]

lImageSize      = [100, 200, 300, 400]
lKernelRadius   = [5, 10, 15]


## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2) #<! Single channel image

In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title(f'Input Image');

In [ ]:
# Pandas Data Frame

numComb  = len(lImageSize) * len(lKernelRadius)
dRunTime = {
    'Image Size'                            : [],
    'Kernel Radius'                         : [],
    'Direct Convolution Run Time [Mili Sec]': [],
    'DFT Convolution Run Time [Mili Sec]'   : [],
    }
for (imgSize, krnlRadius) in itertools.product(lImageSize, lKernelRadius):
    dRunTime['Image Size'].append(imgSize)
    dRunTime['Kernel Radius'].append(krnlRadius)

    mII = ski.transform.resize(mI, (imgSize, imgSize), preserve_range = True)
    mKK = np.random.rand(2 * krnlRadius + 1, 2 * krnlRadius + 1)
    mKK /= np.sum(mKK)

    runTime = timeit.timeit('sp.signal.convolve(mII, mKK, method = "direct")', number = 3, setup = 'import scipy as sp', globals = {'mII': mII, 'mKK': mKK})
    dRunTime['Direct Convolution Run Time [Mili Sec]'].append(runTime * 1e3)

    runTime = timeit.timeit('sp.signal.convolve(mII, mKK, method = "fft")', number = 3, setup = 'import scipy as sp', globals = {'mII': mII, 'mKK': mKK})
    dRunTime['DFT Convolution Run Time [Mili Sec]'].append(runTime * 1e3)

dfRunTime = pd.DataFrame(dRunTime)
dfRunTime

* <font color='red'>(**?**)</font> What about the cases for Gaussian Blur and Box Blur?